In [1]:
from diffusers import StableDiffusionPipeline
import os
import os.path as path

home = os.getenv('HOME')
p = StableDiffusionPipeline.from_single_file(path.join(home, 'civitai', 'animatediff', 'lyriel_v16.safetensors'))

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
/home/kiddos/.pyenv/versions/3.11.4/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [2]:
import torch
from diffusers import MotionAdapter, AnimateDiffPipeline, DDIMScheduler
from diffusers.utils import export_to_gif

adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2")
model_id = "runwayml/stable-diffusion-v1-5"
pipe = AnimateDiffPipeline.from_pretrained(
  model_id,
  vae=p.vae.to(torch.float16),
  unet=p.unet.to(torch.float16),
  tokenizer=p.tokenizer,
  motion_adapter=adapter,
  torch_dtype=torch.float16,
  use_safetensors=True,
)
scheduler = DDIMScheduler.from_config(
  p.scheduler.config
)
pipe.scheduler = scheduler

# pipe.enable_sequential_cpu_offload()
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

The config attributes {'transformer_layers_per_block': 1, 'attention_head_dim': 8, 'class_embed_type': None, 'addition_embed_type': None, 'addition_time_embed_dim': None, 'upcast_attention': None, 'projection_class_embeddings_input_dim': None} were passed to UNetMotionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [3]:
prompt = '(dark shot:1.1), epic realistic, portrait of halo, sunglasses, blue eyes, tartan scarf, white hair by atey ghailan, by greg rutkowski, by greg tocchini, by james gilleard, by joe fenton, by kaethe butcher, gradient yellow, black, brown and magenta color scheme, grunge aesthetic!!! graffiti tag wall background, art by greg rutkowski and artgerm, soft cinematic light, adobe lightroom, photolab, hdr, intricate, highly detailed, (depth of field:1.4), faded, (neutral colors:1.2), (hdr:1.4), (muted colors:1.2), hyperdetailed, (artstation:1.4), cinematic, warm lights, dramatic light, (intricate details:1.1), complex background, (rutkowski:0.66), (teal and orange:0.4)'
negative_prompt = '3d, cartoon, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, artist name, young, loli, elf, 3d, illustration'

generator = torch.Generator("cpu").manual_seed(1261263585)
output = pipe(
  prompt=prompt.strip(),
  num_frames=16,
  guidance_scale=9.0,
  num_inference_steps=30,
  generator=generator,
  clip_skip=2,
)
frames = output.frames[0]
export_to_gif(frames, "/tmp/animatediff.gif")

Token indices sequence length is longer than the specified maximum sequence length for this model (186 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['by greg rutkowski and artgerm, soft cinematic light, adobe lightroom, photolab, hdr, intricate, highly detailed, ( depth of field : 1. 4 ), faded, ( neutral colors : 1. 2 ), ( hdr : 1. 4 ), ( muted colors : 1. 2 ), hyperdetailed, ( artstation : 1. 4 ), cinematic, warm lights, dramatic light, ( intricate details : 1. 1 ), complex background, ( rutkowski : 0. 6 6 ), ( teal and orange : 0. 4 )']


  0%|          | 0/30 [00:00<?, ?it/s]

'/tmp/animatediff.gif'

In [4]:
prompt = 'a forbidden castle high up in the mountains, pixel art, intricate detail'
negative_prompt = '(worst quality, low quality, letterboxed)'

output = pipe(
  prompt=prompt.strip(),
  num_frames=16,
  guidance_scale=9.0,
  num_inference_steps=30,
  clip_skip=2,
)
frames = output.frames[0]
export_to_gif(frames, "/tmp/animatediff2.gif")

  0%|          | 0/30 [00:00<?, ?it/s]

'/tmp/animatediff2.gif'